In [1]:
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig,RunnableGenerator
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf
from typing import Iterator,List,AsyncIterator
from langchain_core.runnables import ConfigurableField,ConfigurableFieldSpec
from langchain_community.chat_message_histories import ChatMessageHistory,RedisChatMessageHistory  # new
from langchain_core.chat_history import BaseChatMessageHistory  # new
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory  # new

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmGPT=ChatOpenAI(model="gpt-4o-mini")

In [4]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            template="You're an assistant who's good at {ability}. Respond in 20 words or fewer"
        ),
        MessagesPlaceholder(
            variable_name="history"
        ),
        HumanMessagePromptTemplate.from_template(
            template="{question}"
        )
    ]
)

In [6]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ability'], input_types={}, partial_variables={}, template="You're an assistant who's good at {ability}. Respond in 20 words or fewer"), additional_kwargs={}),
 MessagesPlaceholder(variable_name='history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]

In [7]:
chain=prompt|llmGemini

<h3>In Memory Chat Message History </h3>

In [8]:
store={}
def getSessionHistory(sessionId:str) -> BaseChatMessageHistory:
    if sessionId not in store:
        store[sessionId]=ChatMessageHistory()
    return store[sessionId]

In [9]:
isinstance(ChatMessageHistory(),BaseChatMessageHistory)

True

In [10]:
chainWithMessageHistory=RunnableWithMessageHistory(
                            chain,
                            get_session_history=getSessionHistory,
                            input_messages_key="question",
                            history_messages_key="history"
                        )

In [11]:
sessionID="ritish"

In [12]:
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"Which is the smallest country in the world?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='Vatican City, an independent city-state enclaved within Rome, Italy, is the smallest country in the world.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-33f6ee80-62bf-4aea-9465-da70f3307b84-0', usage_metadata={'input_tokens': 29, 'output_tokens': 26, 'total_tokens': 55, 'input_token_details': {'cache_read': 0}})

In [13]:
# Remembers
chainWithMessageHistory.invoke(
    input={"ability":"Geography",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='Vatican City is located within Rome, Italy.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-17514669-4c5f-495f-a0bb-4aa144a5bc9c-0', usage_metadata={'input_tokens': 59, 'output_tokens': 11, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}})

In [14]:
store

{'ritish': InMemoryChatMessageHistory(messages=[HumanMessage(content='Which is the smallest country in the world?', additional_kwargs={}, response_metadata={}), AIMessage(content='Vatican City, an independent city-state enclaved within Rome, Italy, is the smallest country in the world.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-33f6ee80-62bf-4aea-9465-da70f3307b84-0', usage_metadata={'input_tokens': 29, 'output_tokens': 26, 'total_tokens': 55, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Where is it located?', additional_kwargs={}, response_metadata={}), AIMessage(content='Vatican City is located within Rome, Italy.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-

In [15]:
store['ritish']

InMemoryChatMessageHistory(messages=[HumanMessage(content='Which is the smallest country in the world?', additional_kwargs={}, response_metadata={}), AIMessage(content='Vatican City, an independent city-state enclaved within Rome, Italy, is the smallest country in the world.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-33f6ee80-62bf-4aea-9465-da70f3307b84-0', usage_metadata={'input_tokens': 29, 'output_tokens': 26, 'total_tokens': 55, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='Where is it located?', additional_kwargs={}, response_metadata={}), AIMessage(content='Vatican City is located within Rome, Italy.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-17514669-4c

In [16]:
# Will not remember due to different sessionID
chainWithMessageHistory.invoke(
    input={"ability":"What?",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":"Tania"}}
    )

AIMessage(content='Please provide more context! I need to know what "it" refers to in order to tell you its location.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-46dd81f1-4c22-4e8b-b884-a4bcf0bbd33e-0', usage_metadata={'input_tokens': 25, 'output_tokens': 24, 'total_tokens': 49, 'input_token_details': {'cache_read': 0}})

<h3> Using Configurable Field Spec to include two parameters: userID and conversationID </h3> 

In [17]:
store={}
def getSessionHistory(userID:str, conversationID:str) -> BaseChatMessageHistory:
    if (userID,conversationID) not in store:
        store[(userID,conversationID)]=ChatMessageHistory()
    return store[(userID,conversationID)]

In [18]:
chainWithMessageHistory=RunnableWithMessageHistory(
                            runnable=chain,
                            get_session_history=getSessionHistory,
                            input_messages_key="question",
                            history_messages_key="history",
                            history_factory_config=[
                                ConfigurableFieldSpec(
                                    id="userID",
                                    annotation=str,
                                    name="User ID",
                                    description="Unique Identifier for the User",
                                    default="",
                                    is_shared=True
                                    ),
                                ConfigurableFieldSpec(
                                    id="conversationID",
                                    annotation=str,
                                    name="Conversation ID",
                                    description="Unique Identifier for the Conversation",
                                    default="",
                                    is_shared=True
                                    )
                            ]
                        )

In [19]:
userID="ritish"
conversationID="1"

In [21]:
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"Which is the smallest country in the world?"},
    config={"configurable":{"userID":userID,"conversationID":conversationID}}
    )

AIMessage(content="Vatican City, located within Rome, Italy, is the world's smallest independent country by both area and population.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-dd15a86a-101e-4564-a8a7-65a25678a72c-0', usage_metadata={'input_tokens': 63, 'output_tokens': 25, 'total_tokens': 88, 'input_token_details': {'cache_read': 0}})

In [22]:
# Remembers
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"Where is it located?"},
    config={"configurable":{"userID":userID,"conversationID":conversationID}}
    )

AIMessage(content="Vatican City is located within the city of Rome, Italy. It's an enclave.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-86e79fe8-9cc7-4554-ac00-c68a5b4ed556-0', usage_metadata={'input_tokens': 92, 'output_tokens': 20, 'total_tokens': 112, 'input_token_details': {'cache_read': 0}})

In [23]:
# Will not remember due to different conversationID
chainWithMessageHistory.invoke(
    input={"ability":"geography",
           "question":"What is it's area?"},
    config={"configurable":{"userID":userID,"conversationID":"2"}}
    )

AIMessage(content='Please tell me what place you are asking about! I need a location to give you its area. ', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-c57aee3b-b61e-4523-b460-8e9df3052d08-0', usage_metadata={'input_tokens': 27, 'output_tokens': 22, 'total_tokens': 49, 'input_token_details': {'cache_read': 0}})

<h3>Use with Redis</h3>

In [24]:
REDIS_URL="redis://localhost:6379/0"

In [25]:
def getMessageHistory(sessionID:str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id=sessionID,url=REDIS_URL)

In [26]:
chainWithMessageHistoryRedis=RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=getMessageHistory,
    input_messages_key="question",
    history_messages_key="history"
)

In [28]:
chainWithMessageHistoryRedis.invoke(
    input={"ability":"geography",
           "question":"Which is the smallest country in the world?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='Vatican City, an independent city-state enclaved within Rome, Italy, is the smallest country in the world.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-c08601d0-388e-4c21-89a0-ff49459e8451-0', usage_metadata={'input_tokens': 29, 'output_tokens': 26, 'total_tokens': 55, 'input_token_details': {'cache_read': 0}})

In [29]:
# Remembers
chainWithMessageHistoryRedis.invoke(
    input={"ability":"geography",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":sessionID}}
    )

AIMessage(content='Vatican City is located within Rome, Italy.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-62b382ee-87ea-4a43-a22c-85d45c1a0cdc-0', usage_metadata={'input_tokens': 59, 'output_tokens': 11, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}})

In [30]:
# Will not remember due to different sessionID
chainWithMessageHistoryRedis.invoke(
    input={"ability":"geography?",
           "question":"Where is it located?"},
    config={"configurable":{"session_id":"Tania"}}
    )

AIMessage(content='To answer that, I need to know what "it" refers to! Please provide more context. ', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-0b92f900-e43a-4f9e-97a8-d80942f44a14-0', usage_metadata={'input_tokens': 25, 'output_tokens': 22, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}})